[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/ai-creator-archive/blob/main/generator_d.ipynb)

In [ ]:
%cd /content

!apt -y install -qq aria2
!pip install diffusers==0.14.0 transformers piexif fold_to_ascii discord ftfy omegaconf -U
!git clone -b v0.14.0 https://github.com/huggingface/diffusers

import os, gc, requests, json, piexif, torch
from diffusers import StableDiffusionPipeline
from PIL.PngImagePlugin import PngInfo
from fold_to_ascii import fold

metadata = PngInfo()

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

root_folder = 'images'

if os.path.exists(f"{root_folder}") == False:
    os.mkdir(f"{root_folder}")
image_folder = max([int(f) for f in os.listdir(f"{root_folder}")], default=0)
if os.path.exists(f"{root_folder}/{image_folder:04}") == False:
    os.mkdir(f"{root_folder}/{image_folder:04}")
name = max([int(f[: f.index(".")]) for f in os.listdir(f"{root_folder}/{image_folder:04}")], default=0)

def generate(discord_token, discord_channel_id, ckpt, discord_user, by, num_inference_steps, guidance_scale, sampler, width, height, prompt, negative_prompt, suffix, image_folder, name):
    model_suffix = ckpt.split("/")[-1].split(".")[-1]
    !rm /content/model.{model_suffix}
    !rm -rf /content/pt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {ckpt} -d /content -o model.{model_suffix}

    if 'pipe' in globals():  
      del pipe
    with torch.no_grad():
      torch.cuda.empty_cache()
    gc.collect()

    !python /content/diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path /content/model.{model_suffix} --dump_path /content/pt
    pipe = StableDiffusionPipeline.from_pretrained("/content/pt", torch_dtype=torch.float16, safety_checker=None).to("cuda")
    
    width = closestNumber(width, 8)
    height = closestNumber(height, 8)
    metadata.add_text("Prompt", f"{prompt}")
    metadata.add_text("by", f"{by}")
    gc.collect()
    real_seed = torch.cuda.initial_seed()
    image = pipe(prompt=prompt, negative_prompt=negative_prompt, num_inference_steps=num_inference_steps, height=height, width=width, guidance_scale=guidance_scale).images[0]
    if(suffix == 'png'):
      image.save(f"{root_folder}/{image_folder:04}/{name:04}.{suffix}", pnginfo=metadata)
    else:
      zeroth_ifd = {piexif.ImageIFD.ImageDescription: f"{fold(prompt)}", piexif.ImageIFD.Make: f"{fold(by)}", piexif.ImageIFD.Model: f"{ckpt}", piexif.ImageIFD.Copyright: f"Attribution 4.0 International (CC BY 4.0)"}
      exif_dict = {"0th": zeroth_ifd}
      exif_bytes = piexif.dump(exif_dict)
      image.save(f"{root_folder}/{image_folder:04}/{name:04}.{suffix}", "JPEG", quality=70, exif=exif_bytes)
    files = {f"{image_folder:04}_{name:04}.{suffix}": open(f"{root_folder}/{image_folder:04}/{name:04}.{suffix}", "rb").read()}
    payload = {"content": f"{prompt}\nSteps: {num_inference_steps}, Sampler: {sampler}, CFG scale: {guidance_scale}, Seed: {real_seed}, Size: {width}x{height}, Model: {ckpt} - {discord_user}"}
    requests.post(f"https://discord.com/api/v9/channels/{discord_channel_id}/messages", data=payload, headers={"authorization": f"Bot {discord_token}"}, files=files)
    os.remove(f"{root_folder}/{image_folder:04}/{name:04}.{suffix}")

In [ ]:
with open("template.json", "r") as file:
    prompts = file.readlines()
for prompt in prompts:
    d = json.loads(prompt)
    name += 1
    generate(d["discord_token"], d["discord_channel_id"], d["ckpt"], "camenduru", d["by"], d["num_inference_steps"], d["guidance_scale"], d["sampler"], d["width"], d["height"], d["prompt"], d["negative_prompt"], d["suffix"], image_folder, name)